# 作业07：CV中的Transformer与迁移学习

## 1. 作业要求
1. 了解ViT的原理和网络结构，将ImageNet上预训练的ViT模型迁移至美食分类问题上。
2. 在README文件中描述具体的迁移学习方法。
3. 可视化模型迁移学习过程中损失函数与准确率的变化趋势，并保存可视化结果。

In [1]:
#导入环境
import os
import zipfile
import random
import json
import sys
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import paddle
from paddle.io import Dataset
from paddle.nn import Conv2D, MaxPool2D, Linear, Dropout, BatchNorm, AdaptiveAvgPool2D, AvgPool2D
import paddle.nn.functional as F
import paddle.nn as nn

## 2. ViT

### 2.1 ViT算法综述
论文地址：[An Image is Worth 16x16 Words:Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

之前的算法大都是保持CNN整体结构不变，在CNN中增加attention模块或者使用attention模块替换CNN中的某些部分。ViT算法中，作者提出没有必要总是依赖于CNN，仅仅使用Transformer结构也能够在图像分类任务中表现很好。

受到NLP领域中Transformer成功应用的启发，ViT算法中尝试将标准的Transformer结构直接应用于图像，并对整个图像分类流程进行最少的修改。具体来讲，ViT算法中，会将整幅图像拆分成小图像块，然后把这些小图像块的线性嵌入序列作为Transformer的输入送入网络，然后使用监督学习的方式进行图像分类的训练。ViT算法的整体结构如 **图1** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/5d33d430cbfe43cb98c6c9926618cb2da8e52318a00341da92b83bc32bedeabb" width = "800"></center>
<center><br>图1：ViT算法结构示意图</br></center>
<br></br>

该算法在中等规模（例如ImageNet）以及大规模（例如ImageNet-21K、JFT-300M）数据集上进行了实验验证，发现：
* Tranformer相较于CNN结构，缺少一定的平移不变性和局部感知性，因此在数据量不充分时，很难达到同等的效果。具体表现为使用中等规模的ImageNet训练的Tranformer会比ResNet在精度上低几个百分点。
* 当有大量的训练样本时，结果则会发生改变。使用大规模数据集进行预训练后，再使用迁移学习的方式应用到其他数据集上，可以达到或超越当前的SOTA水平。

**图2** 为大家展示了使用大规模数据集预训练后的 ViT 算法，迁移到其他小规模数据集进行训练，与使用 CNN 结构的SOTA算法精度对比。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/85d8e641c74648b3b0f34033d137b4ca0f8d9dc7e4a24eeba40e0da48dec96c7" width = "800"></center>
<center><br>图2：ViT模型精度对比</br></center>
<br></br>

图中前3列为不同尺度的ViT模型，使用不同的大规模数据集进行预训练，并迁移到各个子任务上的结果。第4列为BiT算法基于JFT-300M数据集预训练后，迁移到各个子任务的结果。第5列为2020年提出的半监督算法 Noisy Student 在 ImageNet 和 ImageNet ReaL 数据集上的结果。

---

**说明：**

BiT 与 Noisy Student 均为2020年提出的 SOTA 算法。

BiT算法：使用大规模数据集 JFT-300M 对 ResNet 结构进行预训练，其中，作者发现模型越大，预训练效果越好，最终指标最高的为4倍宽、152层深的 $ResNet152 \times 4$。论文地址：[Big Transfer (BiT): General Visual Representation Learning](https://arxiv.org/abs/1912.11370)

Noisy Student 算法：使用知识蒸馏的技术，基于 EfficientNet 结构，利用未标签数据，提高训练精度。论文地址：[Self-training with Noisy Student improves ImageNet classification](https://arxiv.org/abs/1911.04252)

---


接下来，分别看一下ViT算法的各个组成部分。

### 2.2 图像分块嵌入

考虑到之前课程中学习的，Transformer结构中，输入需要是一个二维的矩阵，矩阵的形状可以表示为 $(N,D)$，其中 $N$ 是sequence的长度，而 $D$ 是sequence中每个向量的维度。因此，在ViT算法中，首先需要设法将 $H \times W \times C$ 的三维图像转化为 $(N,D)$ 的二维输入。

ViT中的具体实现方式为：将 $H \times W \times C$ 的图像，变为一个 $N \times (P^2 * C)$ 的序列。这个序列可以看作是一系列展平的图像块，也就是将图像切分成小块后，再将其展平。该序列中一共包含了 $N=HW/P^2$ 个图像块，每个图像块的维度则是 $(P^2*C)$。其中  $P$ 是图像块的大小，$C$ 是通道数量。经过如上变换，就可以将 $N$ 视为sequence的长度了。

但是，此时每个图像块的维度是 $(P^2*C)$，而我们实际需要的向量维度是 $D$，因此我们还需要对图像块进行 Embedding。这里 Embedding 的方式非常简单，只需要对每个 $(P^2*C)$ 的图像块做一个线性变换，将维度压缩为 $D$ 即可。

上述对图像进行分块以及 Embedding 的具体方式如 **图3** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/a1dbbb5ad2384df88f24fc739836c31f42bf2056f78c491cbc5d31b78b933ee3" width = "800"></center>
<center><br>图3：图像分块嵌入示意图</br></center>
<br></br>

具体代码实现如下所示。其中，使用了大小为 $P$ 的卷积来代替对每个大小为 $P$ 图像块展平后使用全连接进行运算的过程。

In [2]:
# 图像分块嵌入模块
class PatchEmbed(nn.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        # 原始大小为int，转为tuple，即：img_size原始输入224，变换后为[224,224]
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        # 图像块的个数
        num_patches = (img_size[1] // patch_size[1]) * \
            (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        # kernel_size=块大小，即每个块输出一个值，类似每个块展平后使用相同的全连接层进行处理
        # 输入维度为3，输出维度为块向量长度
        # 与原文中：分块、展平、全连接降维保持一致
        # 输出为[B, C, H, W]
        self.proj = nn.Conv2D(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            "Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        # [B, C, H, W] -> [B, C, H*W] ->[B, H*W, C]
        x = self.proj(x).flatten(2).transpose((0, 2, 1))
        return x

### 2.3 多头注意力

将图像转化为 $N \times (P^2 * C)$ 的序列后，就可以将其输入到 Tranformer 结构中进行特征提取了。在前面的课程中，我们了解到 Tranformer 结构中最重要的结构就是 Multi-head Attention，即多头注意力结构，如 **图4** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/8566c2480d554506be0c83eb0a0a60736d26aa23b23246bf8db88d59b21a55c9" width = "800"></center>
<center><br>图4：Multi-head Attention 示意图</br></center>
<br></br>


具有2个head的 Multi-head Attention 结构如 **图5** 所示。输入 $a^i$ 经过转移矩阵，并切分生成 $q^{(i,1)}$、$q^{(i,2)}$、$k^{(i,1)}$、$k^{(i,2)}$、$v^{(i,1)}$、$v^{(i,2)}$，然后 $q^{(i,1)}$ 与 $k^{(i,1)}$ 做 attention，得到权重向量 $\alpha$，将 $\alpha$ 与 $v^{(i,1)}$ 进行加权求和，得到最终的 $b^{(i,1)}(i=1,2,…,N)$，同理可以得到 $b^{(i,2)}(i=1,2,…,N)$。接着将它们拼接起来，通过一个线性层进行处理，得到最终的结果。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/4953243f18af450eae3d16181b9a77ce83f4623e414747298b0d7c056c3a6bfe" width = "800"></center>
<center><br>图5：Multi-head Attention结构</br></center>
<br></br>

其中，使用 $q^{(i,j)}$、$k^{(i,j)}$ 与 $v^{(i,j)}$ 计算 $b^{(i,j)}(i=1,2,…,N)$ 的方法是 Scaled Dot-Product Attention。 结构如 **图6** 所示。首先使用每个 $q^{(i,j)}$ 去与 $k^{(i,j)}$ 做 attention，这里说的 attention 就是匹配这两个向量有多接近，具体的方式就是计算向量的加权内积，得到 $\alpha_{(i,j)}$。这里的加权内积计算方式如下所示：

$$ \alpha_{(1,i)} =  q^1 * k^i / \sqrt{d} $$

其中，$d$ 是 $q$ 和 $k$ 的维度，因为 $q*k$ 的数值会随着维度的增大而增大，因此除以 $\sqrt{d}$ 的值也就相当于归一化的效果。

接下来，把计算得到的 $\alpha_{(i,j)}$ 取 softmax 操作，再将其与 $v^{(i,j)}$ 相乘。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/5b3da7158a92461aa1f5cd0bd294a9aba0935bf02d74461b9aa15d48784e8f4e" width = "400"></center>
<center><br>图6：Scaled Dot-Product Attention</br></center>
<br></br>

**想了解注意力机制的更多信息，请参阅[awesome-DeepLearning](https://github.com/paddlepaddle/awesome-DeepLearning) 中的 [注意力机制知识点](https://github.com/PaddlePaddle/awesome-DeepLearning/tree/master/docs/tutorials/deep_learning/model_tuning/attention)。**

具体代码实现如下所示。

In [3]:
# Multi-head Attention
class Attention(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads=8,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop=0.,
                 proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim**-0.5
        # 计算 q,k,v 的转移矩阵
        self.qkv = nn.Linear(dim, dim * 3, bias_attr=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        # 最终的线性层
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        N, C = x.shape[1:]
        # 线性变换
        qkv = self.qkv(x).reshape((-1, N, 3, self.num_heads, C //
                                   self.num_heads)).transpose((2, 0, 3, 1, 4))
        # 分割 query key value
        q, k, v = qkv[0], qkv[1], qkv[2]
        # Scaled Dot-Product Attention
        # Matmul + Scale
        attn = (q.matmul(k.transpose((0, 1, 3, 2)))) * self.scale
        # SoftMax
        attn = nn.functional.softmax(attn, axis=-1)
        attn = self.attn_drop(attn)
        # Matmul
        x = (attn.matmul(v)).transpose((0, 2, 1, 3)).reshape((-1, N, C))
        # 线性变换
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

### 2.4 多层感知机（MLP）

 Tranformer 结构中还有一个重要的结构就是 MLP，即多层感知机，如 **图7** 所示。
 
 <center><img src="https://ai-studio-static-online.cdn.bcebos.com/62a1efbf38bb4c119e89cf277dc2653394a19af9cea5476182406a2ebc0572e9" width = "600"></center>
<center><br>图7：多层感知机</br></center>
<br></br>
 
 多层感知机由输入层、输出层和至少一层的隐藏层构成。网络中各个隐藏层中神经元可接收相邻前序隐藏层中所有神经元传递而来的信息，经过加工处理后将信息输出给相邻后续隐藏层中所有神经元。在多层感知机中，相邻层所包含的神经元之间通常使用“全连接”方式进行连接。多层感知机可以模拟复杂非线性函数功能，所模拟函数的复杂性取决于网络隐藏层数目和各层中神经元数目。多层感知机的结构如 **图8** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/9ada33e2b5134412b2b3dd04dfc0e6e88e932555045147ce99a880f06d69db23" width = "400"></center>
<center><br>图8：多层感知机结构</br></center>
<br></br>

**想了解多层感知机的更多信息，请参阅[awesome-DeepLearning](https://github.com/paddlepaddle/awesome-DeepLearning) 中的 [多层感知机知识点](https://github.com/PaddlePaddle/awesome-DeepLearning/blob/master/docs/tutorials/deep_learning/basic_concepts/multilayer_perceptron.md)。**

具体代码实现如下所示。

In [4]:
# 多层感知机
class Mlp(nn.Layer):
    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        # 输入层：线性变换
        x = self.fc1(x)
        # 应用激活函数
        x = self.act(x)
        # Dropout
        x = self.drop(x)
        # 输出层：线性变换
        x = self.fc2(x)
        # Dropout
        x = self.drop(x)
        return x

### 2.5 基础模块

基于上面实现的 Attention、MLP 和 DropPath 模块就可以组合出 Vision Transformer 模型的一个基础模块，如 **图9** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/5f8f09402d5d442c8f357aa39912865c2253cc7eec374d52821d7f35e566ca67" width = "600"></center>
<center><br>图9：Transformer 基础模块</br></center>
<br></br>

ViT使用了DropPath（Stochastic Depth）来代替传统的Dropout结构，DropPath可以理解为一种特殊的 Dropout。其作用是在训练过程中随机丢弃子图层（randomly drop a subset of layers），而在预测时正常使用完整的 Graph。

In [5]:
def drop_path(x, drop_prob=0., training=False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = paddle.to_tensor(1 - drop_prob)
    shape = (paddle.shape(x)[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + paddle.rand(shape, dtype=x.dtype)
    random_tensor = paddle.floor(random_tensor)
    output = x.divide(keep_prob) * random_tensor
    return output

class DropPath(nn.Layer):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


# 基础模块
class Block(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5):
        super().__init__()
        self.norm1 = eval(norm_layer)(dim, epsilon=epsilon)
        # Multi-head Self-attention
        self.attn = Attention(
            dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop)
        # DropPath
        self.drop_path = DropPath(drop_path) if drop_path > 0. else Identity()
        self.norm2 = eval(norm_layer)(dim, epsilon=epsilon)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim,
                       hidden_features=mlp_hidden_dim,
                       act_layer=act_layer,
                       drop=drop)

    def forward(self, x):
        # Multi-head Self-attention， Add， LayerNorm
        x = x + self.drop_path(self.attn(self.norm1(x)))
        # Feed Forward， Add， LayerNorm
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

### 2.6 定义ViT网络

基础模块构建好后，就可以构建完整的ViT网络了。ViT的完整结构如 **图10** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/60f51da9f9dc477182c9c107d27867b743ff2dcee5fe427fbf81a9d5c0a01806" width = "600"></center>
<center><br>图10：ViT网络结构</br></center>
<br></br>

在实现完整网络结构之前，还需要给大家介绍几个模块：

1. Class Token

可以看到，假设我们将原始图像切分成 $3 \times 3$ 共9个小图像块，最终的输入序列长度却是10，也就是说我们这里人为的增加了一个向量进行输入，我们通常将人为增加的这个向量称为 Class Token。那么这个 Class Token 有什么作用呢？

我们可以想象，如果没有这个向量，也就是将 $N=9$ 个向量输入 Transformer 结构中进行编码，我们最终会得到9个编码向量，可对于图像分类任务而言，我们应该选择哪个输出向量进行后续分类呢？

由于选择9个中的哪个都不合适，所以ViT算法中，提出了一个可学习的嵌入向量 Class Token，将它与9个向量一起输入到 Transformer 结构中，输出10个编码向量，然后用这个 Class Token 进行分类预测即可。

其实这里也可以理解为：ViT 其实只用到了 Transformer 中的 Encoder，而并没有用到 Decoder，而 Class Token 的作用就是寻找其他9个输入向量对应的类别。

2. Positional Encoding

按照 Transformer 结构中的位置编码习惯，这个工作也使用了位置编码。不同的是，ViT 中的位置编码没有采用原版 Transformer 中的 $sincos$ 编码，而是直接设置为可学习的 Positional Encoding。对训练好的 Positional Encoding 进行可视化，如 **图11** 所示。我们可以看到，位置越接近，往往具有更相似的位置编码。此外，出现了行列结构，同一行/列中的 patch 具有相似的位置编码。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/3c2889396cab4790bffb2c23b0954fe552411b45dce14a12a925e2f3ee164790" width = "600"></center>
<center><br>图11：Positional Encoding </br></center>
<br></br>

3. MLP Head

得到输出后，ViT中使用了 MLP Head对输出进行分类处理，这里的 MLP Head 由 LayerNorm 和两层全连接层组成，并且采用了 GELU 激活函数。

具体代码如下所示。

In [6]:
# 参数初始化配置
trunc_normal_ = nn.initializer.TruncatedNormal(std=.02)
zeros_ = nn.initializer.Constant(value=0.)
ones_ = nn.initializer.Constant(value=1.)

# 将输入 x 由 int 类型转为 tuple 类型
def to_2tuple(x):
    return tuple([x] * 2)

# 定义一个什么操作都不进行的网络层
class Identity(nn.Layer):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, input):
        return input

In [7]:
class VisionTransformer(nn.Layer):
    def __init__(self,
                 img_size=384,
                 patch_size=16,
                 in_chans=3,
                 class_dim=1000,
                 embed_dim=768,
                 depth=12,
                 num_heads=12,
                 mlp_ratio=4,
                 qkv_bias=False,
                 qk_scale=None,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5,
                 **args):
        super().__init__()
        self.class_dim = class_dim

        self.num_features = self.embed_dim = embed_dim
        # 图片分块和降维，块大小为patch_size，最终块向量维度为768
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim)
        # 分块数量
        num_patches = self.patch_embed.num_patches
        # 可学习的位置编码
        self.pos_embed = self.create_parameter(
            shape=(1, num_patches + 1, embed_dim), default_initializer=zeros_)
        self.add_parameter("pos_embed", self.pos_embed)
        # 人为追加class token，并使用该向量进行分类预测
        self.cls_token = self.create_parameter(
            shape=(1, 1, embed_dim), default_initializer=zeros_)
        self.add_parameter("cls_token", self.cls_token)
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = np.linspace(0, drop_path_rate, depth)
        # transformer
        self.blocks = nn.LayerList([
            Block(
                dim=embed_dim,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[i],
                norm_layer=norm_layer,
                epsilon=epsilon) for i in range(depth)
        ])

        self.norm = eval(norm_layer)(embed_dim, epsilon=epsilon)

        # Classifier head ## 加几层
        self.head = nn.Linear(embed_dim,
                              512)
        self.head2 = nn.Linear(512, class_dim) if class_dim > 0 else Identity()

        trunc_normal_(self.pos_embed)
        trunc_normal_(self.cls_token)
        self.apply(self._init_weights)
    # 参数初始化
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            zeros_(m.bias)
            ones_(m.weight)
    # 获取图像特征
    def forward_features(self, x):
        B = paddle.shape(x)[0]
        # 将图片分块，并调整每个块向量的维度
        x = self.patch_embed(x)
        # 将class token与前面的分块进行拼接
        cls_tokens = self.cls_token.expand((B, -1, -1))
        x = paddle.concat((cls_tokens, x), axis=1)
        # 将编码向量中加入位置编码
        x = x + self.pos_embed
        x = self.pos_drop(x)
        # 堆叠 transformer 结构
        for blk in self.blocks:
            x = blk(x)
        # LayerNorm
        x = self.norm(x)
        # 提取分类 tokens 的输出
        return x[:, 0]

    def forward(self, x):
        # 获取图像特征
        x = self.forward_features(x)
        # 图像分类
        x.stop_gradient=True # 前面的层不进行训练，即只训练最后一层：
        x = self.head(x)
        x = self.head2(x)

        return x

## 3. 基于ImageNet预训练模型的美食分类

在上文中，已经详细介绍了ViT的算法原理，以及如何使用Paddle实现ViT的模型结构。这一部分为美食分类数据集的导入

In [8]:
'''
参数配置
'''
train_parameters = {
    "input_size": [3, 64, 64],                                #输入图片的shape
    "class_dim": -1,                                          #分类数
    "src_path":"data/data42610/foods.zip",                    #原始数据集路径
    "target_path":"/home/aistudio/data/",                     #要解压的路径
    "train_list_path": "/home/aistudio/data/train.txt",       #train.txt路径
    "eval_list_path": "/home/aistudio/data/eval.txt",         #eval.txt路径
    "readme_path": "/home/aistudio/data/readme.json",         #readme.json路径
    "label_dict":{},                                          #标签字典
    "num_epochs": 15,                                          #训练轮数
    "train_batch_size": 16,                                   #训练时每个批次的大小
    "learning_strategy": {                                    #优化函数相关的配置
        "lr": 0.01                                          #超参数学习率
    } 
}

### 3.1 数据准备

解压模型权重文件。这里使用的模型为PaddleClas套件中预训练的[ViT_base_
patch16_384](https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ViT_base_patch16_384_pretrained.pdparams)。

In [9]:
# 解压权重文件
# !unzip -q -o /home/aistudio/data/data105741/pretrained.zip -d /home/aistudio/work/

### 3.2 图像预处理与批量读取
对美食分类的图像进行预处理，使其满足ViT网络的要求：
1. 图像以Numpy格式存储；
2. 长宽统一裁剪/缩放为384x384
3. **归一化**：将神经网络每层中任意神经元的输入值分布归一化为标准正态分布
4. **通道变换**：图像的数据格式为[H, W, C]（即高度、宽度和通道数），而神经网络使用的训练数据的格式为[C, H, W]，因此需要对图像数据重新排列，例如[384, 384, 3]变为[3, 384, 384]。

In [10]:
def unzip_data(src_path,target_path):
    '''
    解压原始数据集，将src_path路径下的zip包解压至target_path目录下
    '''
    if(not os.path.isdir(target_path + "foods")):     
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()

In [11]:
def get_data_list(target_path,train_list_path,eval_list_path):
    '''
    生成数据列表
    '''
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path=target_path+"foods/"
    class_dirs = os.listdir(data_list_path)  
    #总的图像数量
    all_class_images = 0
    #存放类别标签
    class_label=0
    #存放类别数目
    class_dim = 0
    #存储要写进eval.txt和train.txt中的内容
    trainer_list=[]
    eval_list=[]
    #读取每个类别
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            class_dim += 1
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取类别路径 
            path = data_list_path  + class_dir
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:                                  # 遍历文件夹下的每个图片
                name_path = path + '/' + img_path                       # 每张图片的路径
                if class_sum % 8 == 0:                                  # 每8张图片取一个做验证数据
                    eval_sum += 1                                       # test_sum为测试数据的数目
                    eval_list.append(name_path + "\t%d" % class_label + "\n")
                else:
                    trainer_sum += 1 
                    trainer_list.append(name_path + "\t%d" % class_label + "\n")#trainer_sum测试数据的数目
                class_sum += 1                                          #每类图片的数目
                all_class_images += 1                                   #所有类图片的数目
             
            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = class_dir             #类别名称
            class_detail_list['class_label'] = class_label          #类别标签
            class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
            class_detail.append(class_detail_list)  
            #初始化标签列表
            train_parameters['label_dict'][str(class_label)] = class_dir
            class_label += 1 
            
    #初始化分类数
    train_parameters['class_dim'] = class_dim
    
    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
            
    random.shuffle(trainer_list)
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image) 

    # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path                  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
    with open(train_parameters['readme_path'],'w') as f:
        f.write(jsons)
    print ('生成数据列表完成！')

'''
参数初始化
'''
src_path=train_parameters['src_path']
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
batch_size=train_parameters['train_batch_size']

'''
解压原始数据到指定路径
'''
unzip_data(src_path,target_path)

'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt
with open(train_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(eval_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
    
#生成数据列表   
get_data_list(target_path,train_list_path,eval_list_path)

生成数据列表完成！


In [12]:
import paddle
import paddle.vision.transforms as T
import numpy as np
from PIL import Image


class FoodDataset(paddle.io.Dataset):
    """
    5类food数据集类的定义
    """
    def __init__(self, mode='train'):
        """
        初始化函数
        """
        self.data = []
        with open('data/{}.txt'.format(mode)) as f:
            for line in f.readlines():
                info = line.strip().split('\t')
                if len(info) > 0:
                    self.data.append([info[0].strip(), info[1].strip()])
        self.transforms = T.Compose([
            T.Resize((384, 384)),    # 图片缩放
            T.ToTensor(),                       # 数据的格式转换和标准化、 HWC => CHW            
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])        
    def __getitem__(self, index):
        """
        根据索引获取单个样本
        """
        image_file, label = self.data[index]
        image = Image.open(image_file)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = self.transforms(image)
        return image, np.array(label, dtype='int64')
    def __len__(self):
        """
        获取样本总数
        """
        return len(self.data)

In [13]:
'''
构造数据提供器
'''
train_dataset = FoodDataset(mode='train')
eval_dataset = FoodDataset(mode='eval')
# train_dataload = 
batch_size = train_parameters['train_batch_size']
train_loader = paddle.io.DataLoader(train_dataset, return_list=True, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = paddle.io.DataLoader(eval_dataset, return_list=True, shuffle=True, batch_size=batch_size, drop_last=True)
print(train_dataset.__len__())
print(eval_dataset.__len__())

4375
625


## 4. 迁移学习

将预训练的ViT模型迁移至美食分类问题上。

### 4.1 加载预训练模型

In [14]:
# 实例化模型
model = VisionTransformer(
        patch_size=16,
        class_dim=5,
        embed_dim=768,
        depth=12,
        num_heads=12,
        mlp_ratio=4,
        qkv_bias=True,
        epsilon=1e-6)

# 加载模型参数
params_file_path="/home/aistudio/work/ViT_base_patch16_384_pretrained.pdparams"
model_state_dict = paddle.load(params_file_path)
model.load_dict(model_state_dict)

W1204 12:55:25.674160   120 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1204 12:55:25.679523   120 device_context.cc:465] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1436: UserWarning: Skip loading for head.weight. head.weight receives a shape [768, 1000], but the expected shape is [768, 512].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1436: UserWarning: Skip loading for head.bias. head.bias receives a shape [1000], but the expected shape is [512].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1436: UserWarning: Skip loading for head2.weight. head2.weight is not found in the provided 

### 4.2 调整模型超参数 & 4.3 Fine Tuning

调整模型最后的全连接层以适应美食分类任务（5分类），并完成对模型Fine Tuning的代码。

**参考资料**：
1. [动态图/静态图转换](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/04_dygraph_to_static/index_cn.html#dongtaituzhuanjingtaitu)与[冻结静态图参数](https://www.paddlepaddle.org.cn/documentation/docs/zh/faq/train_cn.html#stop-gradient-true)。
2. [PaddleHub](https://www.paddlepaddle.org.cn/hub)以及[模型打包](https://aistudio.baidu.com/aistudio/projectdetail/1259178)。

In [15]:
model = paddle.Model(model)

In [16]:
##优化器
optim = paddle.optimizer.Adam(learning_rate=train_parameters['learning_strategy']['lr'], parameters=model.parameters())
##为模型设置优化器，loss函数
model.prepare(optim, paddle.nn.CrossEntropyLoss(),paddle.metric.Accuracy())

log_callback = paddle.callbacks.VisualDL(log_dir='logdir_3') # 设置回调visualDL
# model_callback = paddle.callbacks.ModelCheckpoint(save_dir='./model')

model.fit(train_loader, test_loader, epochs=train_parameters['num_epochs'],callbacks=[log_callback],)

model.save('model/model3')

log_callback_test = paddle.callbacks.VisualDL(log_dir='logdir_test3') # 设置回调visualDL

model.evaluate(test_loader,callbacks=[log_callback_test])


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/15


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step  10/273 - loss: 2.3835 - acc: 0.6188 - 302ms/step
step  20/273 - loss: 1.4407 - acc: 0.7031 - 287ms/step
step  30/273 - loss: 4.2922 - acc: 0.7521 - 282ms/step
step  40/273 - loss: 5.5073 - acc: 0.7734 - 279ms/step
step  50/273 - loss: 8.4710 - acc: 0.7812 - 278ms/step
step  60/273 - loss: 1.8887 - acc: 0.7802 - 277ms/step
step  70/273 - loss: 3.4805 - acc: 0.7875 - 276ms/step
step  80/273 - loss: 1.0708 - acc: 0.7945 - 276ms/step
step  90/273 - loss: 4.2988e-06 - acc: 0.8007 - 275ms/step
step 100/273 - loss: 3.2068 - acc: 0.8050 - 275ms/step
step 110/273 - loss: 0.6320 - acc: 0.8136 - 275ms/step
step 120/273 - loss: 1.0373 - acc: 0.8146 - 275ms/step
step 130/273 - loss: 1.8626e-07 - acc: 0.8144 - 275ms/step
step 140/273 - loss: 5.5921 - acc: 0.8161 - 275ms/step
step 150/273 - loss: 1.7276 - acc: 0.8217 - 275ms/step
step 160/273 - loss: 1.1744 - acc: 0.8230 - 274ms/step
step 170/273 - loss: 4.3958e-07 - acc: 0.8239 - 274ms/step
step 180/273 - loss: 4.0285 - acc: 0.8260 - 274ms/ste

{'loss': [0.0], 'acc': 0.9086538461538461}

In [1]:
model.evaluate(test_loader,callbacks=[log_callback_test])

NameError: name 'model' is not defined

In [ ]:
# !visualdl service upload --logdir ./logdir_3/